<a href="https://colab.research.google.com/github/CesarDora/Exploration_of_data_generated_by_passenger_trains/blob/main/Processamento_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact
import numpy as np

#configura google colab pra ler arquivos no drive
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
from datetime import datetime
import pytz
import pandas as pd
import numpy as np
import os

local_timezone = pytz.timezone('America/Sao_Paulo')  # Substitua pelo seu fuso horário, se necessário.

#_____________________________________ Função principal

def minha_funcao(df):
    # Calculando a velocidade, garantindo que seja 0 caso menor que 0
    df['speed (km/h)'] = df['speed (km/h)'].apply(lambda x: 0 if x > 0 else x)

    # Calculando a quantidade de pessoas, garantindo que seja 0 caso menor que 0
    df['pessoas'] = ((df['train_weight (kg)'] - 361790) / 76).round().astype(int)
    df['pessoas'] = df['pessoas'].apply(lambda x: max(x, 0))  # Forçando 0 caso seja menor que 0

    # Calculando o carregamento em %, garantindo que seja 0 caso menor que 0
    df['carregamento %'] = ((df['pessoas'] / 2600) * 100).round().astype(int)
    df['carregamento %'] = df['carregamento %'].apply(lambda x: max(x, 0))

    # Garantindo que a coluna 'head_car_relay' seja do tipo booleano
    df['head_car_relay'] = df['head_car_relay'].astype(bool)

    # Criando coluna 'Velocidade_Media' com valores iniciais como NaN
    df['Velocidade_Media'] = np.nan

    # Loop para calcular a velocidade média entre as linhas consecutivas
    for i in range(len(df)):
        if i == 0:
            velocidade_media = 0.0
        else:
            tempo_anterior = df.at[i - 1, 'datetime']
            tempo_portas = df.at[i - 1, 'Portas_abertas']
            tempo_atual = df.at[i, 'datetime']
            diferenca_tempo = (tempo_atual - tempo_anterior).total_seconds()
            diferenca_tempo -= tempo_portas
            distancia_atual = df.at[i, 'distance (km)']
            distancia_anterior = df.at[i - 1, 'distance (km)']
            velocidade_media = round((distancia_atual - distancia_anterior) / diferenca_tempo * 3600)

            if abs(distancia_atual - distancia_anterior) > 8:
              velocidade_media = 0

        df.at[i, 'Velocidade_Media'] = max(velocidade_media, 0)  # Forçando 0 caso seja menor que 0

    df.at[len(df) - 1, 'Velocidade_Media'] = 0.0

    # Dividindo o DataFrame em dois com base na coluna 'head_car_relay'
    df_true = df[df['head_car_relay'] == True].copy()
    df_false = df[df['head_car_relay'] == False].copy()


    # Salvando os arquivos com base na data e hora
    data_hora_atual = datetime.now(local_timezone)
    nome_arquivo = data_hora_atual.strftime("%Y-%m-%d-%H-%M-%S")
    df_true.to_excel(f"/content/gdrive/My Drive/teste dissertacao/automatico/Consolidado/Graficos/Sentido A {nome_arquivo}.xlsx", index=False)
    df_false.to_excel(f"/content/gdrive/My Drive/teste dissertacao/automatico/Consolidado/Graficos/Sentido B {nome_arquivo}.xlsx", index=False)
    df.to_excel(f"/content/gdrive/My Drive/teste dissertacao/automatico/Consolidado/Graficos/Sentido Original {nome_arquivo}.xlsx", index=False)

    return df_true, df_false, df

#_____________________________________ Onde roda meu código para todos os arquivos da pasta e chama a função

dir = "/content/gdrive/My Drive/teste dissertacao/automatico/Consolidado"
arquivos = os.listdir(dir)

for arquivo in arquivos:
    if arquivo.endswith('.xls') or arquivo.endswith('.xlsx'):
        caminho_arquivo = os.path.join(dir, arquivo)
        df = pd.read_excel(caminho_arquivo)
        df_true, df_false, df_original = minha_funcao(df)